# Chatbot
#### IST664/CIS668 Nature Language Processing
#### Sile Hu, Xiaobin Ning, Henglong So, Ziyun Wang

In this notebook, we will build a AI chatbot in python

At first, we imported the packages needed.

In [11]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
#nltk.download('punkt')
import numpy
import tflearn
import tensorflow
import random

import json
with open('intents.json') as file:
    data = json.load(file)

Extracting Data

In [12]:
words = []
labels = []
docs_x = []
docs_y = []

In [13]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

Word Stemming

In [14]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

Bag of Words

In [15]:
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

In [16]:
training = numpy.array(training)
output = numpy.array(output)

In [17]:
tensorflow.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

In [18]:
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

Training Step: 4999  | total loss: 0.19634 | time: 0.017s
| Adam | epoch: 1000 | loss: 0.19634 - acc: 0.9911 -- iter: 32/38
Training Step: 5000  | total loss: 0.17712 | time: 0.021s
| Adam | epoch: 1000 | loss: 0.17712 - acc: 0.9911 -- iter: 38/38
--
INFO:tensorflow:/Users/henglong/Documents/IST 664 - NLP/project/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


Making Predictions

In [19]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)


def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))

chat()

Start talking with the bot (type quit to stop)!
You: hello
Good to see you again!
You: hi
Hello!
You: what is your name
I'm Bot for IST 664 aka Tech With IST 664.
You: how old are you
18 years young!
You: i want to buy a phone
great! we can definately could help you with that. what phone are you lookin for 
You: samsung
Samsung is a great brand what is your price limit 
You: under 500
We are a few suggesting for you: for under 500 Samsung SGH-A237 Blue No Contract AT&T Cell Phone is the best phone
You: i want to see the score
You can call me Bot for IST 664.
You: quit
